In [ ]:
# Cài đặt các thư viện cần thiết
!pip install gensim scikit-learn matplotlib seaborn pyspark

In [ ]:
import sys
import os
import numpy as np

# Thêm đường dẫn tới thư mục src để import module
sys.path.append(os.path.abspath('src'))

from representations.word_embedder import WordEmbedder

# Khởi tạo embedder
embedder = WordEmbedder("glove-wiki-gigaword-50")

# 1. Lấy vector của một từ
word = "king"
vec = embedder.get_vector(word)
print(f"🔹 Vector của '{word}' (10 phần tử đầu):\n", vec[:10])

# 2. Tính độ tương đồng (similarity)
sim_king_queen = embedder.get_similarity("king", "queen")
sim_king_man = embedder.get_similarity("king", "man")
print(f"\n👑 Similarity(king, queen) = {sim_king_queen:.4f}")
print(f"🧍 Similarity(king, man)   = {sim_king_man:.4f}")

# 3. Tìm các từ đồng nghĩa (most similar)
print("\n🧠 5 từ giống 'computer' nhất:")
for w, score in embedder.get_most_similar("computer", top_n=5):
    print(f"  {w:<15} {score:.4f}")

# 4. Nhúng một văn bản
doc = "The queen rules the country."
vec_doc = embedder.embed_document(doc)
print(f"\n📄 Embedding của văn bản (10 phần tử đầu):\n {vec_doc[:10]}")

In [ ]:
# Chạy script huấn luyện (bạn có thể chạy trực tiếp trong terminal hoặc từ notebook)
# !python test/lab4_embedding_training_demo.py

# Sau khi chạy, ta có thể tải lại model và sử dụng
from gensim.models import Word2Vec

trained_model_path = "results/word2vec_ewt.model"
if os.path.exists(trained_model_path):
    trained_model = Word2Vec.load(trained_model_path)
    print("✅ Model tự huấn luyện đã được tải thành công.")
    
    # Thử tìm từ đồng nghĩa
    print("\n🔹 Các từ giống 'language' nhất (từ model tự huấn luyện):")
    similar_words = trained_model.wv.most_similar("language", topn=5)
    for w, score in similar_words:
        print(f"  {w:<15} {score:.4f}")
else:
    print(f"⚠️ Không tìm thấy model tại {trained_model_path}. Hãy chạy script training trước.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

# Danh sách các từ để trực quan hóa
words = [
    'king', 'queen', 'man', 'woman',
    'france', 'paris', 'germany', 'berlin',
    'apple', 'google', 'microsoft',
    'dog', 'cat', 'pet'
]

# Lấy vector cho các từ
word_vectors = np.array([embedder.get_vector(w) for w in words])

# Giảm chiều bằng PCA
pca = PCA(n_components=2)
vectors_2d = pca.fit_transform(word_vectors)

# Vẽ biểu đồ
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(12, 10))
plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], c='blue', edgecolors='k', alpha=0.6)

# Ghi chú tên các từ
for i, word in enumerate(words):
    plt.annotate(word, (vectors_2d[i, 0], vectors_2d[i, 1]), fontsize=12)
    
plt.title("Trực quan hóa Word Embeddings bằng PCA", fontsize=16)
plt.xlabel("Thành phần chính 1", fontsize=12)
plt.ylabel("Thành phần chính 2", fontsize=12)
plt.grid(True)
plt.show()